In [21]:
import numpy as np
import pandas as pd

import time
from tqdm import tqdm

import glob
import re

In [11]:
file_list = glob.glob('/Users/hwanghyeonjun/Documents/GitHub/data/selenium/채권/*.csv')

In [12]:
file_list

['/Users/hwanghyeonjun/Documents/GitHub/data/selenium/채권/한국 30년 채권 수익율 내역.csv',
 '/Users/hwanghyeonjun/Documents/GitHub/data/selenium/채권/중국 1년 채권 수익율 내역.csv',
 '/Users/hwanghyeonjun/Documents/GitHub/data/selenium/채권/일본 3년 채권 수익율 내역.csv',
 '/Users/hwanghyeonjun/Documents/GitHub/data/selenium/채권/미국 3개월 채권 수익율 내역.csv',
 '/Users/hwanghyeonjun/Documents/GitHub/data/selenium/채권/일본 6년 채권 수익율 내역.csv',
 '/Users/hwanghyeonjun/Documents/GitHub/data/selenium/채권/중국 30년 채권 수익율 내역.csv',
 '/Users/hwanghyeonjun/Documents/GitHub/data/selenium/채권/한국 2년 채권 수익율 내역.csv',
 '/Users/hwanghyeonjun/Documents/GitHub/data/selenium/채권/일본 9년 채권 수익율 내역.csv',
 '/Users/hwanghyeonjun/Documents/GitHub/data/selenium/채권/미국 5년 채권 수익율 내역.csv',
 '/Users/hwanghyeonjun/Documents/GitHub/data/selenium/채권/일본 40년 채권 수익율 내역.csv',
 '/Users/hwanghyeonjun/Documents/GitHub/data/selenium/채권/일본 7년 채권 수익율 내역.csv',
 '/Users/hwanghyeonjun/Documents/GitHub/data/selenium/채권/중국 5년 채권 수익율 내역.csv',
 '/Users/hwanghyeonjun/Documents/GitHub/data/sel

In [13]:
file_names = []
for path in tqdm(file_list):
	file_name = path.split('/')[-1]
	file_names.append(file_name)

100%|██████████| 47/47 [00:00<00:00, 162516.31it/s]


In [14]:
file_names

['한국 30년 채권 수익율 내역.csv',
 '중국 1년 채권 수익율 내역.csv',
 '일본 3년 채권 수익율 내역.csv',
 '미국 3개월 채권 수익율 내역.csv',
 '일본 6년 채권 수익율 내역.csv',
 '중국 30년 채권 수익율 내역.csv',
 '한국 2년 채권 수익율 내역.csv',
 '일본 9년 채권 수익율 내역.csv',
 '미국 5년 채권 수익율 내역.csv',
 '일본 40년 채권 수익율 내역.csv',
 '일본 7년 채권 수익율 내역.csv',
 '중국 5년 채권 수익율 내역.csv',
 '일본 3개월 채권 수익율 내역.csv',
 '한국 3년 채권 수익율 내역.csv',
 '미국 10년물 국채 금리 채권 수익율 내역.csv',
 '일본 20년 채권 수익율 내역.csv',
 '미국 30년 채권 수익율 내역.csv',
 '일본 2년 채권 수익율 내역.csv',
 '일본 10년 채권 수익율 내역.csv',
 '한국 50년 채권 수익율 내역.csv',
 '일본 8년 채권 수익율 내역.csv',
 '일본 15년 채권 수익율 내역.csv',
 '미국 1년 채권 수익율 내역.csv',
 '미국 6개월 채권 수익율 내역.csv',
 '미국 2년 채권 수익율 내역.csv',
 '미국 7년 채권 수익율 내역.csv',
 '한국 5년 채권 수익율 내역.csv',
 '미국 1개월 채권 수익율 내역.csv',
 '일본 1년 채권 수익율 내역.csv',
 '중국 3년 채권 수익율 내역.csv',
 '미국 20년 채권 수익율 내역.csv',
 '일본 30년 채권 수익율 내역.csv',
 '일본 4년 채권 수익율 내역.csv',
 '중국 10년 채권 수익율 내역.csv',
 '일본 9개월 채권 수익율 내역.csv',
 '중국 15년 채권 수익율 내역.csv',
 '일본 6개월 채권 수익율 내역.csv',
 '한국 10년 채권 수익율 내역.csv',
 '미국 3년 채권 수익율 내역.csv',
 '한국 20년 채권 수익율 내역.csv',
 '중국 7년 채권 수

In [26]:
regx = '[가-힣]'

for idx, file in enumerate(file_list):
	df = pd.read_csv(file)
	df['날짜'] = df['날짜'].apply(lambda x: re.sub(regx, '', x))
	df['날짜'] = df['날짜'].apply(lambda x: x.replace(' ', '-'))
	df.sort_values(['날짜'], inplace=True)
	df.to_csv('/Users/hwanghyeonjun/Documents/GitHub/data/selenium/' + file_names[idx], index=False)


47it [00:48,  1.03s/it]


In [24]:
df = pd.read_csv(file_list[0])
df.sort_values(['날짜'])

,날짜,종가,오픈,고가,저가,변동 %
1025,2020년 01월 01일,1.690,1.690,1.690,1.690,0.00%
1024,2020년 01월 02일,1.654,1.713,1.713,1.633,-2.13%
1023,2020년 01월 03일,1.583,1.657,1.665,1.583,-4.29%
1022,2020년 01월 04일,1.595,1.595,1.595,1.595,0.76%
1021,2020년 01월 05일,1.595,1.595,1.595,1.595,0.00%
...,...,...,...,...,...,...
4,2022년 11월 06일,4.100,4.100,4.100,4.100,0.00%
3,2022년 11월 07일,4.176,4.133,4.181,4.127,1.85%
2,2022년 11월 08일,4.155,4.162,4.168,4.120,-0.50%
1,2022년 11월 09일,4.110,4.119,4.138,4.088,-1.08%
